# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd

# Cargar el dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# 1. Filtrar clientes con total_claim_amount > 1000 y respuesta "Yes"
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
print("Clientes con total_claim_amount > 1000 y respuesta 'Yes':")
print(filtered_df.head())

# 2. Promedio de total_claim_amount por policy_type y gender para quienes respondieron "Yes"
yes_df = df[df['response'] == 'Yes']
average_claim = yes_df.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)
print("\nPromedio de total_claim_amount por tipo de póliza y género (response = Yes):")
print(average_claim)

# 3. Número total de clientes por estado (solo mostrar estados con más de 500 clientes)
clientes_por_estado = df['state'].value_counts()
estados_500 = clientes_por_estado[clientes_por_estado > 500]
print("\nEstados con más de 500 clientes:")
print(estados_500)

# 4. Máximo, mínimo y mediana de customer_lifetime_value por education y gender
clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)
print("\nEstadísticas de customer_lifetime_value por nivel educativo y género:")
print(clv_stats)


Clientes con total_claim_amount > 1000 y respuesta 'Yes':
     unnamed:_0 customer       state  customer_lifetime_value response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     coverage             education effective_to_date employmentstatus gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [2]:
# Asegurarse de que la columna de fecha esté en formato datetime
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# Crear columna de mes
df['month'] = df['effective_to_date'].dt.month

# Contar pólizas por estado y mes
policies_by_state_month = df.groupby(['state', 'month']).size().reset_index(name='policy_count')

# Total de pólizas por estado
total_by_state = policies_by_state_month.groupby('state')['policy_count'].sum().reset_index()

# Seleccionar los 3 estados con más ventas
top_states = total_by_state.sort_values(by='policy_count', ascending=False).head(3)['state']

# Filtrar los datos para solo esos estados
top_data = policies_by_state_month[policies_by_state_month['state'].isin(top_states)]

# Crear tabla con filas = estados y columnas = meses
pivot_policies = top_data.pivot(index='state', columns='month', values='policy_count').fillna(0)

print(pivot_policies)


/var/folders/fn/p25q3z154s1c65ncyj7gy2mr0000gn/T/ipykernel_28442/3461639315.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])


month          1     2
state                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


In [3]:
# Total de clientes por canal
total_by_channel = df['sales_channel'].value_counts()

# Clientes que respondieron "Yes" por canal
yes_by_channel = df[df['response'] == 'Yes']['sales_channel'].value_counts()

# Calcular porcentaje de respuesta por canal
response_rate = (yes_by_channel / total_by_channel * 100).round(2)

# Mostrar el resultado como DataFrame
response_table = pd.DataFrame({
    'Total Customers': total_by_channel,
    'Responded Yes': yes_by_channel,
    'Response Rate (%)': response_rate
})

print(response_table)


               Total Customers  Responded Yes  Response Rate (%)
sales_channel                                                   
Agent                     4121            742              18.01
Branch                    3022            326              10.79
Call Center               2141            221              10.32
Web                       1626            177              10.89
